In [1]:
import sys
module_path = "./MasaCtrl"
sys.path.append(module_path)



In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from tqdm import tqdm
# from einops import rearrange, repeat
from omegaconf import OmegaConf

from diffusers import DDIMScheduler, ControlNetModel, StableDiffusionPipeline, StableDiffusionControlNetPipeline

from MasaCtrl.masactrl.diffuser_utils import MasaCtrlPipeline, MasaCtrlControlNetPipeline
from MasaCtrl.masactrl.masactrl_utils import AttentionBase
from MasaCtrl.masactrl.masactrl_utils import regiter_attention_editor_diffusers
from MasaCtrl.masactrl.masactrl import MutualSelfAttentionControl

from torchvision.utils import save_image
from torchvision.io import read_image
from pytorch_lightning import seed_everything

from torchvision.transforms import ToPILImage

import glob

torch.cuda.set_device(0)  # set the GPU device

/mnt/hdd/hbchoe/anaconda3/envs/sheeprl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
import random
import datetime as dt

import numpy as np
import torch

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

start_code = torch.randn([1, 4, 64, 64], device=device)
start_code_masa = start_code.expand(2, -1, -1, -1) # expand to batch size 2

In [5]:
noun_list = ["boy", "girl", "man", "woman", "child", 
 "farmer", "soldier", "firefighter", "pirate", "basketball player"]
pose_list = ["dancing", "flexing", 'jumping', 'laying', 'tposing']

In [6]:
import torch
from diffusers import DDIMScheduler
from p2p import Editor, CrossAttnCtrl
from utils import image_grid, latent_to_image

In [9]:
pose_path = "/mnt/hdd/hbchoe/workspace/MasaCtrl/sampling/pose_selected"
source_path = "/mnt/hdd/hbchoe/workspace/MasaCtrl/sampling/p2p/source"
edit_path = "/mnt/hdd/hbchoe/workspace/MasaCtrl/sampling/p2p/edit"

# model initialization
model_path = "stable-diffusion-v1-5/stable-diffusion-v1-5"
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
# controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-openpose")
# model = StableDiffusionPipeline.from_pretrained(model_path, controlnet=controlnet, scheduler=scheduler, safety_checker=None, cross_attention_kwargs={"scale": 0.5}).to(device)


SD = Editor.from_pretrained(
    model_path, scheduler=scheduler, torch_dtype=torch.float32,
    cache_dir='.', processor=CrossAttnCtrl(),
).to(device)

# images = SD(
#     prompt=prompts, prompt_target=prompts_edit, latents=start_code, 
#     num_inference_steps=50,
# ).images

# images[0].save(f'{source_path}/prompt.png')
# images[1].save(f'{edit_path}/prompt.png')


for noun in noun_list:    
    for pose in pose_list:
        prompts = f"highly detailed, {noun}, standing, facing camera, full body portrait, full-length portrait"
        print(f"Source prompt: {prompts}")

        prompts_edit = f"highly detailed, {noun}, {pose}, facing camera, full body portrait, full-length portrait"
        print(f"Edit prompt: {prompts}")

        images = SD(
            prompt=prompts, prompt_target=prompts_edit, latents=start_code, 
            num_inference_steps=50,
        ).images

        images[0].save(f'{source_path}/{noun}_{pose}.png')
        images[1].save(f'{edit_path}/{noun}_{pose}.png')


/mnt/hdd/hbchoe/anaconda3/envs/sheeprl/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/mnt/hdd/hbchoe/anaconda3/envs/sheeprl/lib/python3.9/site-packages/diffusers/pipelines/stable_diffusion/pipeline_stable_diffusion.py:115: FutureWarning: The configuration file of this scheduler: DDIMScheduler {
  "_class_name": "DDIMScheduler",
  "_diffusers_version": "0.16.0",
  "beta_end": 0.012,
  "beta_schedule": "scaled_linear",
  "beta_start": 0.00085,
  "clip_sample": false,
  "

Source prompt: highly detailed, boy, standing, facing camera, full body portrait, full-length portrait
Edit prompt: highly detailed, boy, standing, facing camera, full body portrait, full-length portrait


100%|██████████| 50/50 [00:19<00:00,  2.53it/s]


Source prompt: highly detailed, boy, standing, facing camera, full body portrait, full-length portrait
Edit prompt: highly detailed, boy, standing, facing camera, full body portrait, full-length portrait


100%|██████████| 50/50 [00:19<00:00,  2.52it/s]


...

100%|██████████| 50/50 [00:19<00:00,  2.50it/s]


Source prompt: highly detailed, basketball player, standing, facing camera, full body portrait, full-length portrait
Edit prompt: highly detailed, basketball player, standing, facing camera, full body portrait, full-length portrait


100%|██████████| 50/50 [00:19<00:00,  2.50it/s]


Source prompt: highly detailed, basketball player, standing, facing camera, full body portrait, full-length portrait
Edit prompt: highly detailed, basketball player, standing, facing camera, full body portrait, full-length portrait


100%|██████████| 50/50 [00:19<00:00,  2.50it/s]
